<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

# [Machine Learning Tutorial](https://github.com/wikistat/MLTraining): IoT and Human Activity Recognition (HAR)
## Analyse  de signaux  issus d'un *smartphone* 
## Utilisation des librairies <a href="http://scikit-learn.org/stable/#"><img src="http://scikit-learn.org/stable/_static/scikit-learn-logo-small.png" style="max-width: 100px; display: inline" alt="Scikit-Learn"/></a> en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a> et <a href="https://keras.io/"><img src="https://s3.amazonaws.com/keras.io/img/keras-logo-2018-large-1200.png" style="max-width: 100px; display: inline" alt="Keras"/></a> 


### Résumé
Cas d'usage de [reconnaissance d'activités humaines](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) à partir des enregistrements de signaux (gyroscope, accéléromètre) issus d'un *objet connecté*: un simple smartphone. Les données sont analysées pour illustrer les principales étapes communes en *science des données* et appliquables à des signaux physiques échantillonnés. Visualisation des signaux bruts afin d'évaluer les difficultés posées par ce type de données; exploration ([analyse en composantes principales](http://wikistat.fr/pdf/st-m-explo-acp.pdf), [analyse factorielle discriminante](http://wikistat.fr/pdf/st-m-explo-acp.pdf)) des données transformées (*features*) ou *métier* calculées à partir des signaux; prévision de l'activité à partir des données métier par la plupart des méthodes linéaires dont: [régression logistique](http://wikistat.fr/pdf/st-m-app-rlogit.pdf), [SVM](http://wikistat.fr/pdf/st-m-app-svm.pdf) et non linéaires; prévision de l'activité à partir des signaux bruts par [réseau de neurones](http://wikistat.fr/pdf/st-m-app-rn.pdf) de type perceptron multicouches. 

##  1 Introduction
### 1.1  Objectif général
L'objectif est de reconnaître l'activité d'un individu porteur d'un smartphone qui enregistre un ensemble de signaux issus du gyroscope et de l'accéléromètre embarqués et ainsi connectés. Une base de données d'apprentissage a été construite expérimentalement. Un ensemble de porteurs d'un smartphone ont produit une activité déterminée pendant un laps de temps prédéfini tandis que des signaux étaient enregistrés. Les données sont issues de la communauté qui vise la reconnaissance d'activités humaines (*Human activity recognition, HAR*). Voir à ce propos l'[article](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-11.pdf) relatant un colloque de 2013.  L'analyse des données associée à une identification d'activité en temps réel, ne sont pas abordées.

Les données publiques disponibles ont été acquises, décrites et partiellement analysées par [Anguita et al. (2013)](https://www.icephd.org/sites/default/files/IWAAL2012.pdf). Elles sont accessibles sur le [dépôt](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) de l'University California Irvine (UCI) consacré à l'apprentissage machine.

L'archive contient les données brutes: accélérations échantillonnnées à 64 htz pendant 2s. Les accélérations en x, y, et z, chacune de 128 colonnes, celles en y soustrayant la gravité naturelle ainsi que les accélérations angulaires (gyroscope) en x, y, et z soit en tout 9 fichiers. Le choix d'une puissance de 2 pour la fréquence d'échantillonnage permet l'exécution efficace d'algorithmes de transformée de Fourier ou en ondelettes.


### 1.2 Déroulement
Une première visualisation et exploration des signaux bruts montre (section 2) que ceux-ci sont difficiles à analyser; les classes d'activité y sont en effet mal caractérisées. La principale cause est l'absence de synchronisation des débuts d'activité; le déphasage des signaux apparaît alors comme un bruit ou artefact très préjudiciable à la bonne discrimination des activités sur la base d'une distance euclidienne usuelle ($L_2$). C'est la raison pour laquelle, [Anguita et al. (2013)](https://www.icephd.org/sites/default/files/IWAAL2012.pdf) proposent de calculer un ensemble de transformations ou caractéristiques (*features*) classiques du traitement du signal: variance, corrélations, entropie, décompositions de Fourier... Ce sont alors $p=561$ variables qui sont considérées et explorées dans la section 3. L'[analyse en composantes principales](http://wikistat.fr/pdf/st-m-explo-acp.pdf) et surtout l'[analyse factorielle discriminante](http://wikistat.fr/pdf/st-m-explo-acp.pdf) montrent les bonnes qualités discriminatoires de ces données "métier" issues d'une connaissance experte des signaux. La section 4 exploite ces variables métier et montre que des modèles statistiques élémentaires car linéaires (régression logistique, analyse discriminante) ou qu'un algorithme classique de machine à vecteurs supports (SVM) utilisant un simple noyau linéaire conduisent à d'excellentes prévisions au contraire d'algorihtmes non linaires sophistiqués (*random forest*).

Néanmoins, faire calculer en permanence des transformations sophistiquées  n'est pas une solution viable pour la batterie d'un objet embarqué connecté. L'algorithme candidat doit pouvoir produire une solution intégrable (cablée) dans un cicuit, comme c'est par exemple le cas des puces dédiées à la reconnaissance faciale. C'est l'objet de la section 5: montrer la faisabilité d'une solution basée sur les seuls signaux bruts; solution mettant en oeuvre des réseaux de neurones.


### 1.3 Environnement logiciel
Pour être exécuté, ce calepin (*jupyter notebook*) nécessite l'installation de Python 3 via par exemple le site  [Anaconda](https://conda.io/docs/user-guide/install/download.html). Les algorihtmes d'exploration et d'apprentissage statistique utilisés sont disponibles dans la librairie [`Scikit-learn`](http://scikit-learn.org/stable/). 


# <FONT COLOR="Red">Episode 1</font>

## 2 Etude préalable des signaux bruts
### 2.1 Source

Les données sont celles  du dépôt de l'[UCI](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones). Elle peuvent  être préalablement téléchargées en cliquant [ici](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip).

Chaque enregistrement ou unité statistique ou instance est labellisée avec **6 activités**: debout, assis, couché, marche, monte ou descend un escalier. Chaque jeu de données est partagé en une partie échantillon d'apprentissage et une partie échantillon test. L'échantillon test n'est utilisé que pour évaluer et comparer les qualités de prévision des principales méthodes. Il est conservé en l'état afin de rendre les comparaisons possibles avec les résultats de la littérature. Il s'agit donc d'un problème de *classification supervisée* (6 classes) avec $n=10299$ échantillons pour l'apprentissage, 2947 pour le test.

Les données contiennent deux jeux de dimensions différentes:

1. Jeu multidimensionel: un individus est constitué de 9 Séries Temporelles de *dimensions* $(n, 128, 9)$.
2. Jeu unidimensionnel: Les 9 Séries Temporelles sont concaténées pour constituer un vecteur de 128*9 = 1152 variables de *dimensions* $(n, 1152)$.

*N.B.* La structure des données est nettement plus complexe que celles couramment étudiées dans le [dépôt Wikistat](https://github.com/wikistat/). Le code a été structuré en une séquence de fonctions afin d'en faciliter la compréhension. L'outil *calepin* atteint ici des limites pour la réalisation de codes complexes.


### 2.2 Importation des principales librairies.

In [ ]:
import pandas as pd
import numpy as np
import copy
import random
import itertools
%matplotlib inline
import matplotlib.pyplot as plt
import time

### 2.3 Structurer les données
Définir le chemin d'accès aux données puis les fonctions utiles.

In [ ]:
# Attention: le chemin ci-dessous doit être adapté au contexte
DATADIR_UCI = './UCI HAR Dataset'
# Liste des noms des fichiers afin d'automatiser la lecture.
SIGNALS = [ "body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z", "total_acc_x", "total_acc_y", "total_acc_z"]

# Fonctions permettant de lire la séquence des fichiers avant de restructurer les données 
# dans le fortmat recherché.
def my_read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

def load_signal(data_dir, subset, signal):
    filename = data_dir+'/'+subset+'/Inertial Signals/'+signal+'_'+subset+'.txt'
    x = my_read_csv(filename).values
    return x 

def load_signals(data_dir, subset, flatten = False):
    signals_data = []
    for signal in SIGNALS:
        signals_data.append(load_signal(data_dir, subset, signal)) 
    if flatten :
        X = np.hstack(signals_data)
    else:
        X = np.transpose(signals_data, (1, 2, 0))    
    return X 

def load_y(data_dir, subset, dummies = False):
    filename = data_dir+'/'+subset+'/y_'+subset+'.txt'
    y = my_read_csv(filename)[0]
    if dummies:
        Y = pd.get_dummies(y).values
    else:
        Y = y.values
    return Y 

Lecture des données

In [ ]:
#Multidimensional Data
X_train, X_test = load_signals(DATADIR_UCI, 'train'), load_signals(DATADIR_UCI, 'test')
# Flattened Data
X_train_flatten, X_test_flatten = load_signals(DATADIR_UCI, 'train', flatten=True), load_signals(DATADIR_UCI, 'test', flatten=True)

# Label Y
Y_train_label, Y_test_label = load_y(DATADIR_UCI, 'train', dummies = False), load_y(DATADIR_UCI, 'test', dummies = False)
#Dummies Y (For Keras)
Y_train_dummies, Y_test_dummies = load_y(DATADIR_UCI, 'train', dummies = True), load_y(DATADIR_UCI, 'test', dummies = True)

N_train = X_train.shape[0]
N_test = X_test.shape[0]

Vérification des dimensions afin de s'assurer de la bonne lecture des fichiers.

In [ ]:
print("Dimension")
print("Données Multidimensionelles, : " + str(X_train.shape))
print("Données Unimensionelles, : " + str(X_train_flatten.shape))
print("Vecteur réponse (scikit-learn) : " + str(Y_train_label.shape))
print("Matrice réponse(Keras) : " + str(Y_train_dummies.shape))

### 2.4 Visualisations
Cette phase est essentielle à la bonne compréhension des données, de leur structure et donc des problèmes qui vont être soulevés par la suite. La visualisation est très élémentaire d'un point de vue méthodologique mais nécessite des compétences plus élaborées en Python et donc des fonctions préalables. 
#### Fonctions utiles

In [ ]:
# Liste des couleurs
CMAP = plt.get_cmap("Accent")
# Liste des types de signaux
SIGNALS = ["body_acc x", "body_acc y", "body_acc z", 
                "body_gyro x", "body_gyro y", "body_gyro z", 
               "total_acc x", "total_acc y", "total_acc z"] 
# Dictionnaire en clair des activités expérimentées (contexte supervisé)
ACTIVITY_DIC = {1 : "WALKING",
2 : "WALKING UPSTAIRS",
3 : "WALKING DOWNSTAIRS",
4 : "SITTING",
5 : "STANDING",
6 : "LAYING"}
labels = ACTIVITY_DIC.values()

# Fonction pour le tracé d'un signal
def plot_one_axe(X, fig, ax, sample_to_plot, cmap):
    for act,Xgb in X.groupby("Activity"):
        Xgb_first_values = Xgb.values[:sample_to_plot,:-1]
        x = Xgb_first_values[0]
        ax.plot(x, linewidth=1, color=cmap(act-1), label = label_dic[act])
        for x in Xgb_first_values[1:]:
            ax.plot(x, linewidth=1, color=cmap(act-1))
def plot_one_axe_shuffle(X, fig, ax, sample_to_plot, cmap):
    plot_data = []
    for act,Xgb in X.groupby("Activity"):
        Xgb_first_values = Xgb.values[:sample_to_plot,:-1]
        x = Xgb_first_values[0]
        ax.plot(x, linewidth=1, color=cmap(act-1), label = label_dic[act])
        for x in Xgb_first_values[1:]:
            plot_data.append([x,cmap(act-1)])
    random.shuffle(plot_data)
    for x,color in plot_data:
        ax.plot(x, linewidth=1, color=color)

#### Tracés de tous les signaux
Tous les signaux sont tracés par type en superposant les activités.

In [ ]:
sample_to_plot = 50
index_per_act = [list(zip(np.repeat(act, sample_to_plot), np.where(Y_train_label==act)[0][:sample_to_plot])) for act in range(1,7)]
index_to_plot = list(itertools.chain.from_iterable(index_per_act))
random.shuffle(index_to_plot)

fig = plt.figure(figsize=(15,10))
for isignal in range(9):
    ax = fig.add_subplot(3,3,isignal+1)
    for act , i in index_to_plot:
        ax.plot(range(128), X_train[i,:,isignal],color=CMAP(act-1), linewidth=1)
        ax.set_title(SIGNALS[isignal])
plt.tight_layout()

**Remarque :** Apprécier la difficulté à distinguer les activités au sein d'un même signal.

### 3.3 Par signal 
Le seul signal "acélération en" x est tracé en distinguant les activités.

In [ ]:
sample_to_plot = 10
isignal = 1
index_per_act_dict = dict([(act, np.where(Y_train_label==act)[0][:sample_to_plot]) for act in range(1,7)])


fig = plt.figure(figsize=(15,8), num=SIGNALS[isignal])
for act , index in index_per_act_dict.items():
    ax = fig.add_subplot(3,2,act)
    for x in X_train[index]:
        ax.plot(range(128), x[:,0],color=CMAP(act-1), linewidth=1)
    ax.set_title(ACTIVITY_DIC[act])
plt.tight_layout()

**Q.** Quelle est l'activité qui semble se distinguer facilement des autres ? 

**Q.** Observer les signaux d'une activité, par exemple `Walking upstairs`. Qu'est-ce qui fait que, pour ces signaux ou courbes, une métrique Euclidienne classique ($L_2$) est inopérante ? 

C'est la raison pour laquelle il sera important de décomposer notamment les signaux dans le domaine des fréquences. 


### 3.4 [Analyse en composantes principales](http://wikistat.fr/pdf/st-m-explo-acp.pdf)
Il est important de se faire une idée précise de la structure des données.  Une analyse en composantes principales est adaptée à cet objectif. 
#### Remarques
   - L'ACP n'est pas réduite sur ces données car cette transformation est sans effet sur la piètre qualité des graphiques.
   - L'ACP basée sur une métrique Euclidienne usuelle ne fait que confirmer les difficultés précedemment identifiées et l'absence de pouvoir discriminant des données brutes au sens de cette métrique; cette exploration n'est pas approfondies sur ces données. En revanche un autre [calepin](https://github.com/wikistat/Exploration/blob/master/HumanActivityRecognition/Explo-Python-Har-brutes.ipynb) détaille une analyse factorielle discriminante mais avec la même conclusion.


La fonction définie ci-après affiche un nuage de points dans un plan factoriel.

In [ ]:
def plot_pca(X_R, ytrain, fig, ax, nbc, nbc2, label_dic=ACTIVITY_DIC, cmaps = plt.get_cmap("Accent")
):
    for i in range(6):
        xs = X_R[ytrain==i+1,nbc-1]
        ys = X_R[ytrain==i+1, nbc2-1]
        label = label_dic[i+1]
        color = cmaps(i)
        ax.scatter(xs, ys, color=color, alpha=.8, s=10, label=label)
        ax.set_xlabel("PC%d : %.2f %%" %(nbc,pca.explained_variance_ratio_[nbc-1]*100), fontsize=15)
        ax.set_ylabel("PC%d : %.2f %%" %(nbc2,pca.explained_variance_ratio_[nbc2-1]*100), fontsize=15)

#### ACP sur un type de signal

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
# Choix du signal
isignal = 4
signal = SIGNALS[isignal]
print("ACP Sur signal : " +signal)
X_c = pca.fit_transform(X_train[:,:,isignal])

Représentation des parts de variance expliquée par les premières composantes principales.

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(1,2,1)
ax.bar(range(10), pca.explained_variance_ratio_[:10]*100, align='center',
        color='grey', ecolor='black')
ax.set_xticks(range(10))
ax.set_ylabel("Variance")
ax.set_title("", fontsize=35)
ax.set_title(u"Pourcentage de variance expliquée \n par les premières composantes", fontsize=20)

ax = fig.add_subplot(1,2,2)
box=ax.boxplot(X_c[:,0:10],whis=100)
ax.set_title(u"Distribution des premières composantes", fontsize=20)

fig.suptitle(u"Résultat ACP sur Signal : " + signal, fontsize=25)

**Attention**: les diagrammes boîtes sont très perturbés par les distributions des composantes avec une très forte concentration autour de 0 et énormément de valeurs atypiques. D'où l'utilisation du paramètre `whis=100` pour rallonger les moustaches.

**Q.** Que sont les graphes ci-dessus ? Quelles interprétations ou absence d'interprétation en tirer ?

**Représentation du premier plan factoriel :**

In [ ]:
fig = plt.figure(figsize=(10,10), )
ax = fig.add_subplot(1,1,1)
plot_pca(X_c, Y_train_label,fig ,ax ,1 ,2)

Les autres plans factoriels ne sont guère plus informatifs.
#### Sur tous les signaux
Tous les signaux sont concaténés à plat en un seul signal.

In [ ]:
pca = PCA()
print("ACP Sur tous les signaux")
X_c = pca.fit_transform(X_train_flatten)

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(1,2,1)
ax.bar(range(10), pca.explained_variance_ratio_[:10]*100, align='center',
        color='grey', ecolor='black')
ax.set_xticks(range(10))
ax.set_ylabel("Variance")
ax.set_title("", fontsize=35)
ax.set_title(u"Pourcentage de variance expliqué \n des premières composantes", fontsize=20)

ax = fig.add_subplot(1,2,2)
box=ax.boxplot(X_c[:,0:10],whis=100)
ax.set_title(u"Distribution des premières composantes", fontsize=20)

fig.suptitle(u"Résultat ACP", fontsize=25)

In [ ]:
fig = plt.figure(figsize=(10,10), )
ax = fig.add_subplot(1,1,1)
plot_pca(X_c, Y_train_label,fig ,ax ,1 ,3)

**Q.** Quelle activité semble néanmoins facile à identifier ?

## 3 Exploration des données "métier"
### 3.1 Les données
L'[archive de l'UCI]() contient également deux fichiers `train` et `test` des 561 caractéristiques (*features*) ou variables "métier" calculées dans les domaines temporels et fréquentiels par transformation des signaux bruts.

Voici une liste indicative des variables calculées sur chacun des signaux bruts ou couples de signaux:

Name|Signification
-|-
mean | Mean value
std | Standard deviation
mad | Median absolute value
max | Largest values in array
min | Smallest value in array
sma | Signal magnitude area
energy | Average sum of the squares
iqr | Interquartile range
entropy | Signal Entropy
arCoeff | Autorregresion coefficients
correlation | Correlation coefficient
maxFreqInd | Largest frequency component
meanFreq | Frequency signal weighted average
skewness | Frequency signal Skewness
kurtosis | Frequency signal Kurtosis
energyBand | Energy of a frequency interval
angle | Angle between two vectors

#### Lecture des données métier

In [ ]:
# Lecture des données d'apprentissage
# Attention, il peut y avoir plusieurs espaces comme séparateur dans le fichier
Xtrain=pd.read_csv("X_train.txt",sep='\s+',header=None)
Xtrain.head()

In [ ]:
# Variable cible
ytrain=pd.read_csv("y_train.txt",sep='\s+',header=None,names=['y'])
# Le type dataFrame est inutile et même gênant pour la suite
ytrain=ytrain["y"]

In [ ]:
# Lecture des données de test
Xtest=pd.read_csv("X_test.txt",sep='\s+',header=None)
Xtest.shape
ytest=pd.read_csv("y_test.txt",sep='\s+',header=None,names=['y'])
ytest=ytest["y"]

### 3.2 [Analyse en composantes principales](http://wikistat.fr/pdf/st-m-explo-acp.pdf)
Fonction graphique pour les plans factoriels.

In [ ]:
def plot_pca(X_R,fig,ax,nbc,nbc2):
    for i in range(6):
        xs = X_R[ytrain==i+1,nbc-1]
        ys = X_R[ytrain==i+1, nbc2-1]
        label = ACTIVITY_DIC [i+1]
        color = cmaps(i)
        ax.scatter(xs, ys, color=color, alpha=.8, s=1, label=label)
        ax.set_xlabel("PC%d : %.2f %%" %(nbc,pca.explained_variance_ratio_[nbc-1]*100), fontsize=10)
        ax.set_ylabel("PC%d : %.2f %%" %(nbc2,pca.explained_variance_ratio_[nbc2-1]*100), fontsize=10)

Calcul de la matrice des composantes principales. C'est aussi un changement (transformation) de base; de la base canonique dans la base des vecteurs propres. 

In [ ]:
pca = PCA()
X_c = pca.fit_transform(Xtrain)

#### Valeurs propres ou variances des composantes principales
Représentation de la décroissance des valeurs propres, les variances des variables ou composantes principales.

In [ ]:
plt.plot(pca.explained_variance_ratio_[0:10])
plt.show()

Un graphique plus explicite décrit les distribution de ces composantes par des diagrames boîtes; seules les premières sont affichées. 

In [ ]:
plt.boxplot(X_c[:,0:10])
plt.show()

Commenter la décroissance des variances, le choix éventuel d'une dimension ou nombre de composantes à retenir sur les 561.
#### Représentation des individus ou "activités" en ACP
Projection dans les principaux plans factoriels.

In [ ]:
cmaps = plt.get_cmap("Accent")
fig = plt.figure(figsize= (20,20))
count = 0
for nbc, nbc2,count in [(1,2,1), (2,3,2), (3,4,3), (1,3,4), (2,4,5), (1,4,7)] :
    ax = fig.add_subplot(3,3,count)
    plot_pca(X_c, fig,ax,nbc,nbc2)
plt.legend(loc='upper right', bbox_to_anchor=(1.8, 0.5), markerscale=10)
plt.show()

**Q.** Commenter la séparation des deux types de situation par le premier axe.

**Q.** Que dire sur la forme des nuages ?

**Q.** Que dire sur la plus ou moins bonne séparation des classes ?
#### Représentation des variables en ACP
Lecture des libellés des variables et constitution d'une liste. Souci de la grande dimension (561), les représentations ne sont guère exploitables. 

In [ ]:
with open('features.txt', 'r') as content_file:
    featuresNames = content_file.read()
columnsNames = list(map(lambda x : x.split(" ")[1],featuresNames.split("\n")[:-1]))

Graphe des variables illisible en mettant les libellés en clair. Seule une * est représentée.

In [ ]:
# coordonnées des variables
coord1=pca.components_[0]*np.sqrt(pca.explained_variance_[0])
coord2=pca.components_[1]*np.sqrt(pca.explained_variance_[1])
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1, 1, 1)
for i, j in zip(coord1,coord2, ):
    plt.text(i, j, "*")
    plt.arrow(0,0,i,j,color='r')
plt.axis((-1.2,1.2,-1.2,1.2))
# cercle
c=plt.Circle((0,0), radius=1, color='b', fill=False)
ax.add_patch(c)
plt.show()

Identification des variables participant le plus au premier axe. Ce n'est pas plus clair! Seule la réprésentation des individus apporte finalement des éléments de compréhension.

In [ ]:
print(np.array(columnsNames)[abs(coord1)>.6])

### 3.3 [Analyse Factorielle Discriminante (AFD)](http://wikistat.fr/pdf/st-m-explo-afd.pdf)
#### Principe
L'ACP ne prend pas en compte la présence de la variable qualitative à modéliser contrairement à l'analyse factorielle discriminante (AFD) adaptés à ce contexte "supervisé" puisque l'activité est connue sur un échantillon d'apprentissage. L'AFD est une ACP des barycentres des classes munissant l'espace des individus d'une métrique spécifique dite de *Mahalanobis*. Métrique définie par l'inverse de la matrice de covariance intraclase. L'objectif est alors de visualiser les capacités des variables à discriminer les classes.

La librairie `scikit-learn` ne propose pas de fonction spécifique d'analyse factorielle discriminante mais les coordonnées des individus dans la base des vecteurs discriminants sont obtenues comme résultats de l'analyse discriminante linéaire décisionnnelle. Cette dernière sera utilisée avec une finalité prédictive dans un deuxième temps (autre calepin). 

Les résultats de la fonction `LinearDiscriminantAnalysis` de `scikit-learn` sont identiques à ceux de la fonction `lda` de R. Elle est donc utilisée strictement de la même façon.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
method = LinearDiscriminantAnalysis() 
lda=method.fit(Xtrain,ytrain)
X_r2=lda.transform(Xtrain)

**Q.** Que signifie le *warning* ? Quel traitement faudrait-t-il mettre en oeuvre pour utiliser une analyse discriminante décisionnelle en modélisation ou apprentissage ?

#### Représentation des individus en AFD


In [ ]:
fig = plt.figure(figsize= (20,20))
count = 0
for nbc, nbc2,count in [(1,2,1), (2,3,2), (3,4,3), (1,3,4), (2,4,5), (1,4,7)] :
    ax = fig.add_subplot(3,3,count)
    plot_pca(X_r2, fig,ax,nbc,nbc2)
plt.legend(loc='upper right', bbox_to_anchor=(1.8, 0.5), markerscale=10)
plt.show()

**Q.** Que dire de la séparation des classes ? Sont-elles toutes séparables deux-à-deux ?

**Q.** Que dire de la forme des nuages notamment dans le premier plan ?

Comme pour l'ACP, la représentation trop complexe des variables n'apporterait rien.

### 3.4 [Classification non supervisée](http://wikistat.fr/pdf/st-m-explo-classif.pdf)
Cette section n'est pas utile puisque les classes sont connues. Néanmoins, une approche générale des l'étude de signaux relatant des activités humaines non identifiées *a priori* nécessiterait cette phase de classification non supervisée ou *clustering*. Cette étape permet simplement ici  d'illustrer le comportement d'un algorithme de classification non supervisée classique. La matrice de confusion des classes obtenues avec celles connues permet d'en évaluer les performances. 
#### $k$*-means*
Attention, il est nécessaire de centrer et réduire les variables avant d'exécuter un algorithme de classification non supervisé.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
tps1 = time.perf_counter()
X = StandardScaler().fit_transform(Xtrain)
km=KMeans(n_clusters=6)
km.fit(Xtrain)
tps2 = time.perf_counter()
print("Temps execution Kmeans :", (tps2 - tps1))

In [ ]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(ytrain, km.labels_)[1:7,0:6].T, columns=labels)

**Q.** Que dire de l'efficacité d'une approche non supervisée pour catégoriser les activités ?

# <FONT COLOR="Red">Episode 2</font>

## 4 Prévision de l'activité à partir des variables "métier"
Plusieurs méthodes sont successivement testées dans ce calepin : SVM, analyse discriminate décisionnelle, $k$ plus proches voisins, forêts aléatoires, réseaux de neurones... Nous commençons par la régression logistique  pour la prévision du comportement.

### 4.1 [Régression logistique](http://wikistat.fr/pdf/st-m-app-rlogit.pdf)

####  Principe
Une méthode statistique ancienne mais finalement efficace sur ces données. La régression logistique est adaptée à la prévision d'une variable binaire. Dans le cas multiclasse, la fonction logistique de la librairie `Scikit-learn` estime *par défaut* **un modèle par classe**: une classe contre les autres. 

La probabilité d'appartenance d'un individu à une classe est modélisée à l'aide d'une combinaison linéaire des variables explicatives. Pour transformer une combinaison linéaire à valeur dans $R$ en une probabilité à valeurs dans l'intervalle $[0, 1]$, une fonction de forme sigmoïdale est appliquée.  Ceci donne: $$P(y_i=1)=\frac{e^{Xb}}{1+e^{Xb}}$$ ou, c'est équivalent, une décomposition linéaire du *logit* ou *log odd ratio* de  $P(y_i=1)$:  $$\log\frac{P(y_i=1)}{1-P(y_i=1)}=Xb.$$


####  Estimation du modèle sans optimisation
Le modèle est estimé sans chercher à raffiner les valeurs de certains paramètres (pénalisation). Ce sera fait dans un deuxième temps. Le paramètre de choix du *solver* est précisé car le choix par défaut (`lbfgs`) semble converger moins vite. Une comparaison systématique des différentes options (`liblinear, lbfgs, saga, sag, newton-cg`) serait bienvenue en association avec le choix de modèle lorsque le nombre de classes est plus grand que 2: fonction perte multinomiale ou un modèle binomial par classe.

In [ ]:
from sklearn.linear_model import LogisticRegression
ts = time.time()
method = LogisticRegression(solver='liblinear',multi_class='auto')
method.fit(Xtrain,ytrain)
score = method.score(Xtest, ytest)
ypred = method.predict(Xtest)
te = time.time()

#### Prévision de l'activité de l'échantillon test
Une fois le modèle estimé, l'erreur de prévision est évaluée, sans biais optimiste, sur un autre échantillon, dit échantillon test, qui n'a pas participé à l'apprentissage du modèle.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(ytest, ypred), index = labels, columns=labels)

**Q.** Quelles sont les classes qui restent difficiles à discriminer ?

**Q.** Commenter la qualité des résultats obtenus. Sont-ils cohérents avec l'approche exploratoire ?

#### Optimisation du modèle par pénalisation Lasso
*Attention* l'exécution est un peu longue... cette optimisation peut être omise en première lecture.

In [ ]:
# Optimisation du paramètre de pénalisation
# grille de valeurs
from sklearn.model_selection import GridSearchCV
ts = time.time()
param=[{"C":[0.5,1,5,10,12,15,30]}]
logit = GridSearchCV(LogisticRegression(penalty="l1",solver='liblinear', 
                                        multi_class='auto'), param,cv=10,n_jobs=-1)
logitOpt=logit.fit(Xtrain, ytrain)  
# paramètre optimal
logitOpt.best_params_["C"]
te = time.time()
print("Temps : %d secondes" %(te-ts))

In [ ]:
print("Meilleur score = %f, Meilleur paramètre = %s" % (logitOpt.best_score_,logitOpt.best_params_))

In [ ]:
yChap = logitOpt.predict(Xtest)
# matrice de confusion
logitOpt.score(Xtest, ytest)

In [ ]:
pd.DataFrame(confusion_matrix(ytest, yChap), index = labels, columns=labels)

**Q.** L'amélioration est-elle bien significative au regard du temps de calcul ?

**Q.** Déterminer les variables sélectionnées par la méthode LASSO. 

### 4.2 [Analyse discriminante linéaire](http://wikistat.fr/pdf/st-m-app-add.pdf)
**Q.** Que dire de l'optimisation de cette méthode ? Celle-ci est proposée dans une librairie de R mais pas disponible en Python.

**Q.** L'analyse discriminante quadratique pose quelques soucis. Pourquoi ?


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
ts = time.time()
method = LinearDiscriminantAnalysis()
method.fit(Xtrain,ytrain)
score = method.score(Xtest, ytest)
ypred = method.predict(Xtest)
te = time.time()
score

In [ ]:
print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(ytest, ypred), index = labels, columns=labels)

### 4.3 [*K* plus proches voisins](http://wikistat.fr/pdf/st-m-app-add.pdf)

Cette méthode peut être vue comme un cas particulier d'analyse discriminante avec une estimation locale des fonctions de densité conditionnelle. 

**Q.** Combien de voisins sont utilisés par défaut ? Optimiser ce paramètre. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
ts = time.time()
method = KNeighborsClassifier(n_jobs=-1)
method.fit(Xtrain,ytrain)
score = method.score(Xtest, ytest)
ypred = method.predict(Xtest)
te = time.time()
t_total = te-ts

In [ ]:
print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(ytest, ypred), index = labels, columns=labels)

### 4.4 [SVM linéaire](http://wikistat.fr/pdf/st-m-app-svm.pdf)


Le nombre max d'itérations a été très sensiblement augmenté (1000 par défaut) sans pour autant améliorer la performance. Le cas multi-classes est traité en considérant une classe contre les autres donc 6 modèles.

**Q.** Utiliser le paramètre par défaut puis faire varier le paramètre de pénalisation. 

In [ ]:
from sklearn.svm import LinearSVC
ts = time.time()
method = LinearSVC(max_iter=20000)
method.fit(Xtrain,ytrain)
score = method.score(Xtest, ytest)
ypred = method.predict(Xtest)
te = time.time()
t_total = te-ts

In [ ]:
print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(ytest, ypred), index = labels, columns=labels)

### 4.5 [SVM avec noyau gaussien](http://wikistat.fr/pdf/st-m-app-svm.pdf)

Apprentissage avec les valeurs par défaut puis optimisation des paramètres.

**Q.** Quels sont les paramètres à optimiser ?

In [ ]:
from sklearn.svm import SVC
ts = time.time()
method = SVC(gamma='auto')
method.fit(Xtrain,ytrain)
score = method.score(Xtest, ytest)
ypred = method.predict(Xtest)
te = time.time()

In [ ]:
print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(ytest, ypred), index = labels, columns=labels)

**Q.** Quelle procédure est exécutée ci-après et dans quel but ?

*Attention*: l'exécution est un peu longue et peut être omise en preière lecture.

In [ ]:
ts = time.time()
param=[{"C":[4,5,6],"gamma":[.01,.02,.03]}]
svm= GridSearchCV(SVC(),param,cv=10,n_jobs=-1)
svmOpt=svm.fit(Xtrain, ytrain)
te = time.time()
te-ts

In [ ]:
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (svmOpt.best_score_,svmOpt.best_params_))

**Q.** Comparer les deux approches par SVM (linéaire et radiale): temps de calcul et performance.

### 4.6 [*Random forest*](http://wikistat.fr/pdf/st-m-app-agreg.pdf)

**Q.** Quel serait le paramètre à optimiser ?

In [ ]:
from sklearn.ensemble import RandomForestClassifier
ts = time.time()
method = RandomForestClassifier(n_estimators=200,n_jobs=-1)
method.fit(Xtrain,ytrain)
score = method.score(Xtest, ytest)
ypred = method.predict(Xtest)
te = time.time()
t_total = te-ts

In [ ]:
print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(ytest, ypred), index = labels, columns=labels)

### 4.8 Combinaison de modèles
Les formes des nuages de chaque classe observées dans le premier plan de l'analyse en composantes principales montrent que la structure de covariance n'est pas identique dans chaque classe. Cette remarque suggèrerait de s'intéresser à l'analyse discriminante quadratique mais celle-ci bloque sur l'estimation des six matrices de covariance et de leur inverse. Néanmoins il semble que, plus précisément, deux groupes se distinguent : les classes actives (marcher, monter ou descendre un escalier) d'une part et les classes passives (couché, assis, debout) d'autre part et, qu'à l'intérieur de chaque groupe les variances sont assez similaires. 

Cette situation suggère de construire une décision en deux étapes ou hiérarchique:
1. Régression logistique séparant les activités passives *vs.* actives,
2. Un modèle spécifique à chacune des classes précédentes, par exemple des SVM à noyau gaussien.

Une telle construction hiérarchique de modèles aboutit à une précision supérieure à 97%.

**Exercice :** Programmer une telle approche en utilisant les capacités de python pour réaliser un *pipeline*.

# <FONT COLOR="Red">Episode 3</font>

## 5 Prévision de l'activité à partir des signaux bruts
### 5.1 Introduction
Comme expliqué en introduction, le calcul des nombreuses transformations des données est bien trop consommateur des ressources de la batterie d'un objet connecté. Cette section se propose d'utiliser les seuls signaux bruts pour faire apprendre un algorithme et parmi les algorithmes possibles, seuls les réseaux de neurones pouvant être "cablés" dans un circuit sont pris en compte. Nous utilisons ici un réseau de type perceptron multicouche. 


### 5.2 Perceptron à une couche cachée
#### Librairies 

In [ ]:
import tensorflow as tf
import tensorflow.keras.models as km 
import tensorflow.keras.layers as kl

In [ ]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}
def my_confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

#### Définition du réseau 
Une couche cachée, une couche de reformattage puis une couche de sortie à 6 classes. Le nombre de neurones (50) sur la couche cachée a été optimisé par ailleurs. Le nombre d'epochs  et la taille des batchs devraient être optimisés surtout dans le cas d'utilisation d'une carte GPU.

Remarquer le nombre de paramètres à estimer.

In [ ]:
epochs=20
batch_size=32
n_hidden = 50

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

model_base_mlp =km.Sequential()
model_base_mlp.add(kl.Dense(n_hidden, input_shape=(timesteps, input_dim),  activation = "relu"))
model_base_mlp.add(kl.Reshape((timesteps*n_hidden,) , input_shape= (timesteps, n_hidden)  ))
model_base_mlp.add(kl.Dense(n_classes, activation='softmax'))

model_base_mlp.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model_base_mlp.summary()

In [ ]:
t_start = time.time()
model_base_mlp.fit(X_train,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_mlp.evaluate(X_test, Y_test_dummies)[1] 
print("\nScore With Simple MLP on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_mlp = {"time_learning" : t_learning, "score" : score}
base_mlp_prediction = model_base_mlp.predict(X_test)

my_confusion_matrix(Y_test_dummies, base_mlp_prediction)

**Q.** L'ajout d'une couche cachée permet-il d'améliorer les résultats ? 

Quelques efforts supplémentaires permettraient sans doute de gagner quelques points sur la précision du résultat mais... attention au sur-apprentissage si le même échantillon test est toujours utilisé. 

En 5ième année, dans l'UF High Dimensional and Deep Learning (HDDL), un réseau concolutionnel (CNN) sur les signaux permettra d'obtenir de meilleures performances, proches de celles obtenues sur les données métier. 

